In [2]:
from pathlib import Path
import yaml
import pandas as pd
from collections import Counter
import random
from sklearn.model_selection import KFold
import datetime
import shutil
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.data.split import autosplit

# Generating Feature Vectors for Object Detection Dataset

In [3]:
# retrieve dataset labels
dataset_path = Path('./datasets/v3-2')
labels = sorted((dataset_path.rglob('labels/train/*.txt')))

# read class labels
yaml_file = dataset_path / 'data.yaml'

with open(yaml_file, encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())


index = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=index)

for label in labels:
    lbl_counter = Counter()

    with open(label) as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ", 1)[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

labels_df

/tmp/ipykernel_23688/3694895945.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`


,0
20250121_154454,1.0
20250121_154511,1.0
20250121_154952,1.0
20250121_155000,1.0
20250121_155005,1.0
...,...
frame_001096,0.0
frame_001097,0.0
frame_001098,0.0
frame_001099,0.0


# K-Fold Dataset Split

In [8]:
random.seed(42)
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=index, columns=folds)

for i, (train, val) in enumerate(kfolds, start=1):
    folds_df.loc[labels_df.iloc[train].index, f"split_{i}"] = "train"
    folds_df.loc[labels_df.iloc[val].index, f"split_{i}"] = "val"

fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio


supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.absolute().as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )


for image, label in tqdm(zip(images, labels), total=len(images), desc="Copying files"):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

Copying files: 100%|██████████| 2132/2132 [01:16<00:00, 27.81it/s] 


# Save Records (Optional)

In [9]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

# Train YOLO using K-Fold Data Splits

In [10]:
weights_path = "runs/detect/yolo11-nano/weights/best.pt"
model = YOLO(weights_path, task="detect")

results = {}

# Define your additional arguments here
batch = 16
project = "kfold_demo"
epochs = 2

for k, dataset_yaml in enumerate(ds_yamls):
    model = YOLO(weights_path, task="detect")
    results[k] = model.train(
        data=dataset_yaml, epochs=epochs, batch=batch, project=project, name=f"fold_{k + 1}"
    )  # include any additional train arguments

New https://pypi.org/project/ultralytics/8.3.143 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.114 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2060, 5742MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/yolo11-nano/weights/best.pt, data=datasets/v3-2/2025-05-24_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=fold_12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=F

train: Scanning /home/golde/Documents/Projects/hoop-tracking-training/datasets/v3-2/2025-05-24_5-Fold_Cross-val/split_1/train/labels... 1705 images, 27 backgrounds, 0 corrupt: 100%|██████████| 1705/1705 [00:06<00:00, 247.64it/s]

train: New cache created: /home/golde/Documents/Projects/hoop-tracking-training/datasets/v3-2/2025-05-24_5-Fold_Cross-val/split_1/train/labels.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 412.5±247.9 MB/s, size: 2037.3 KB)


val: Scanning /home/golde/Documents/Projects/hoop-tracking-training/datasets/v3-2/2025-05-24_5-Fold_Cross-val/split_1/val/labels... 427 images, 13 backgrounds, 0 corrupt: 100%|██████████| 427/427 [00:02<00:00, 167.34it/s]

...

        2/2      2.45G     0.2367     0.2093     0.9067         21        640: 100%|██████████| 107/107 [00:20<00:00,  5.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  5.28it/s]

                   all        426        419      0.997          1      0.995      0.991



2 epochs completed in 0.014 hours.
Optimizer stripped from kfold_demo/fold_5/weights/last.pt, 5.5MB
Optimizer stripped from kfold_demo/fold_5/weights/best.pt, 5.5MB

Validating kfold_demo/fold_5/weights/best.pt...
Ultralytics 8.3.114 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2060, 5742MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:03<00:00,  4.36it/s]


                   all        426        419      0.997      0.995      0.995      0.995
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to kfold_demo/fold_5
